In [1]:
import nltk, math, codecs
from gensim.models import Doc2Vec
from nltk.cluster.kmeans import KMeansClusterer
import re
import pymorphy2
from datetime import datetime

fname = 'noStopLemma_PV-DBOW_wrd-vec_1it_2win_6mincount_alpha25-25_sz80.model'

model = Doc2Vec.load(fname)

morph = pymorphy2.MorphAnalyzer()

C:\Anaconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    txt = f.read().split('\n')
stw = set(txt)

In [3]:
#lemmatized test

start_time = datetime.now()
corpus = codecs.open('test_headlines_short.txt', mode="r", encoding="utf-8")
lines = corpus.read().lower().split('\r\n')
lemm_lines = []
for line in lines:
    fixed = ''.join([x if x.isalnum() or x.isspace() else " " for x in line ]).split()
    fixedNoStop = []
    for fix in fixed:
        if fix not in stw:
            fix = morph.parse(fix)[0].normal_form
            fixedNoStop.append(fix)
    lemm_lines.append(fixedNoStop)

    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:07.156179


In [ ]:
print(lines[:10])

In [10]:
NUM_CLUSTERS = 35 #25 #35 #25 #20 #40 #30 

def preprocess(str):
    # remove links
    str = re.sub(r'http(s)?:\/\/\S*? ', "", str)
    return str


def preprocess_document(text):
    #text = preprocess(text)
    fixedNoStop = []
    fixed = ''.join([x if x.isalnum() or x.isspace() else " " for x in text ]).split()
    for fix in fixed:
        if fix not in stw:
            fix = morph.parse(fix)[0].normal_form
            fixedNoStop.append(fix)
    return fixedNoStop

start_time = datetime.now()

#data = <sparse matrix that you would normally give to scikit>.toarray()

corpus = codecs.open('test_headlines_short.txt', mode="r", encoding="utf-8")
lines = corpus.read().lower().split('\r\n')
count = len(lines)

vectors = []

print("inferring vectors")
duplicate_dict = {}
used_lines = []
for i, t in enumerate(lines):
    if t not in duplicate_dict:#i % 2 == 0 and
        duplicate_dict[t] = True
        used_lines.append(t)
        vectors.append(model.infer_vector(preprocess_document(t)))

print("done")



kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=20)
assigned_clusters = kclusterer.cluster(vectors, assign_clusters=True)
print('Cluster assigning done!')
    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

inferring vectors
done
Cluster assigning done!
Duration: 0:11:50.410457


In [18]:
len(used_lines)

2210

In [4]:
clustersizes = []

def distanceToCentroid():
    for i in range(0, NUM_CLUSTERS):
        clustersize = 0
        for j in range(0, len(assigned_clusters)):
            if (assigned_clusters[j] == i):
                clustersize+=1
        clustersizes.append(clustersize)
        dist = 0.0
        centr = kclusterer.means()[i]
        for j in range(0, len(assigned_clusters)):
            if (assigned_clusters[j] == i):
                dist += pow(nltk.cluster.util.cosine_distance(vectors[j], centr),2)/clustersize
        dist = math.sqrt(dist)
        print("distance cluster: "+str(i)+" RMSE: "+str(dist)+" clustersize: "+str(clustersize))

def nClosestToCentroid(cluster_id, n):
    #clustersize = len(get_titles_by_cluster(cluster_id))
    centr = kclusterer.means()[cluster_id]
    distances = []
    for j in range(0, len(assigned_clusters)):
        if (assigned_clusters[j] == cluster_id):
            distances.append((used_lines[j], nltk.cluster.util.cosine_distance(vectors[j], centr)))
    distances = sorted(distances, key=lambda tup: tup[1])
    return distances[:n]


def get_titles_by_cluster(id):
    list = []
    for x in range(0, len(assigned_clusters)):
        if (assigned_clusters[x] == id):
            list.append(used_lines[x])
    return list

def get_topics(titles):
    from collections import Counter
    words = [preprocess_document(x) for x in titles]
    words = [word for sublist in words for word in sublist]
    #filtered_words = [word for word in words if word not in stw]
    count = Counter(words)
    print(count.most_common()[:5])


def cluster_to_topics(id):
    get_topics(get_titles_by_cluster(id))

In [12]:
for clstr in range(NUM_CLUSTERS):
    cluster_to_topics(clstr)

[('теракт', 19), ('задержать', 15), ('мужчина', 9), ('террорист', 8), ('боевик', 8)]
[('рубль', 23), ('миллиард', 16), ('миллион', 12), ('доллар', 8), ('евро', 8)]
[('путин', 29), ('глава', 15), ('президент', 14), ('мид', 13), ('встреча', 13)]
[('россия', 16), ('рост', 8), ('считать', 8), ('россиянин', 7), ('рейтинг', 6)]
[('нефть', 14), ('рассказать', 9), ('добыча', 8), ('рынок', 7), ('роснефть', 6)]
[('выборы', 23), ('президент', 16), ('выбор', 13), ('франция', 11), ('макрон', 10)]
[('россия', 6), ('рассказать', 5), ('путин', 5), ('дом', 4), ('трамп', 3)]
[('каталония', 12), ('мадрид', 5), ('полиция', 5), ('призвать', 4), ('протест', 4)]
[('москва', 20), ('рейс', 8), ('пассажир', 7), ('метро', 7), ('станция', 6)]
[('заявить', 29), ('россия', 22), ('сша', 22), ('ес', 16), ('путин', 14)]
[('сми', 19), ('сообщить', 16), ('москва', 8), ('сообщение', 8), ('источник', 8)]
[('крым', 14), ('украина', 8), ('киев', 7), ('россия', 6), ('слово', 5)]
[('экс', 16), ('глава', 16), ('министр', 9), (

In [ ]:
get_titles_by_cluster(8)

In [ ]:
distanceToCentroid()

In [6]:
len(assigned_clusters)

2210

In [14]:
len(get_titles_by_cluster(5))

70

In [15]:
nClosestToCentroid(5, 23)

[('макрона во втором туре выборов во франции поддержат 61% избирателей, показал опрос',
  0.1524389442078623),
 ('"дух поражения" и "война против всех": как прошли дебаты макрона и ле пен',
  0.18256925746362251),
 ('ле пен обгонит макрона в первом туре выборов, показал опрос',
  0.18304331194618217),
 ('в киргизии лидера парламентской оппозиции выдвинули кандидатом в президенты',
  0.18750982616175227),
 ('в германии проходят парламентские выборы', 0.18943013591974256),
 ('народная партия лидирует на выборах в австрии', 0.19061378682680263),
 ('трамп поздравил макрона с победой на выборах во франции',
  0.19440853465870833),
 ('макрон представил программу на выборах президента франции',
  0.19690478219429131),
 ('блок меркель лидирует после подсчета голосов в 227 округах',
  0.19743194481812909),
 ('ле пен проголосовала на выборах президента. прямая трансляция',
  0.20134079074080091),
 ('врио ярославского губернатора лидирует на региональных выборах',
  0.20745874946175824),
 ('премь

In [17]:
#30 кластеров, 25 повторений
for clstr in range(NUM_CLUSTERS):
    n = round(len(get_titles_by_cluster(clstr))/3)
    print(nClosestToCentroid(clstr, n))
    print('####\n')

[('в мадриде арестовали мужчину, угрожавшего прохожим ножом', 0.18803858060409928), ('в саудовской аравии казнили экстремистов, осужденных за атаки в эль-катифе', 0.22662346368779185), ('задержанный по подозрению в подготовке теракта в вене признал связь с иг', 0.23413742065635235), ('в австрии задержали мужчину, подражавшего гитлеру', 0.23465056975415322), ('путин осудил теракт в лондоне', 0.25050976672067571), ('в армении разыскивают гражданина сша по обвинению в организации терактов', 0.25123188110173633), ('в чечне торговцев оружием подозревают в связях с террористами', 0.25155692459679357), ('в каталонии задержали мужчину, ранившего полицейских', 0.25211831896391734), ('сми заявили, что исполнитель теракта в стамбуле хорошо знал место преступления', 0.25240714146630228), ('напавший на полицию у нотр-дама был психически болен, рассказал его брат', 0.25583105109677329), ('роухани обвинил саудовскую аравию в поддержке террористов в йемене и сирии', 0.25858286655782992), ('на юго-вост

In [19]:
#40 кластеров, 25 повторений
for clstr in range(NUM_CLUSTERS):
    n = round(len(get_titles_by_cluster(clstr))/3)
    print(nClosestToCentroid(clstr, n))
    print('####\n')

[('на ставрополье завели дело на мужчину, сломавшего челюсть полицейскому', 0.14176739013688389), ('полицейские поймали оленя, гулявшего по санкт-петербургу', 0.15239008334232163), ('в краснодаре ищут водителя, обстрелявшего полицейского', 0.16349142806703321), ('в чувашии полицейский насмерть сбил пешехода', 0.16985088657740122), ('в мадриде арестовали мужчину, угрожавшего прохожим ножом', 0.17513262143428299), ('в петербурге задержали подростка, стрелявшего из пневматики по трамваю', 0.19109080727562455), ('в зеленограде мужчина устроил стрельбу в кафе', 0.1925648995948116), ('под тамбовом пьяный полицейский насмерть сбил пешехода', 0.19410421896002328), ('техасский полицейский в сочельник застрелил шестилетнего мальчика', 0.2053180430686723), ('норвежец заснял удар молнии, разгромившей его задний двор', 0.20743359394979877), ('в москве водитель bmw протаранил автомобиль полиции и скрылся', 0.21191168469478183), ('мэр николаева сбежал от полицейских через окно', 0.21451926107868946),

In [20]:
for clstr in range(NUM_CLUSTERS):
    cluster_to_topics(clstr)

[('полицейский', 11), ('мужчина', 11), ('задержать', 6), ('водитель', 5), ('полиция', 5)]
[('терроризм', 12), ('борьба', 10), ('теракт', 4), ('нью', 3), ('британия', 3)]
[('рубль', 18), ('миллиард', 12), ('миллион', 8), ('неделя', 7), ('россия', 7)]
[('задержать', 20), ('подозревать', 9), ('теракт', 9), ('убийство', 7), ('полицейский', 7)]
[('рассказать', 12), ('рост', 9), ('путин', 9), ('россия', 8), ('нефть', 6)]
[('россия', 15), ('заявить', 13), ('сша', 10), ('ес', 9), ('урегулирование', 9)]
[('путин', 10), ('сфера', 8), ('рассказать', 7), ('заявить', 7), ('обсудить', 6)]
[('выборы', 13), ('выбор', 10), ('макрон', 9), ('франция', 6), ('тур', 6)]
[('ирак', 8), ('боевик', 8), ('турция', 7), ('иго', 7), ('поддержка', 6)]
[('взрыв', 18), ('пострадавший', 13), ('теракт', 12), ('число', 8), ('метро', 7)]
[('каталония', 9), ('полиция', 7), ('оппозиция', 5), ('мадрид', 5), ('митинг', 5)]
[('взрыв', 10), ('москва', 7), ('дом', 6), ('звонок', 6), ('минирование', 6)]
[('россия', 12), ('сша', 5

In [22]:
#20 кластеров, 25 повторений
for clstr in range(NUM_CLUSTERS):
    n = round(len(get_titles_by_cluster(clstr))/3)
    print(nClosestToCentroid(clstr, n))
    print('####\n')

[('якушев: регионы намерены реализовать до 2022 г инвестпроекты на 18 трлн руб', 0.20378756601462733), ('биткоин превысил отметку в десять тысяч долларов', 0.20655793076222195), ('волков: урожайность зерновых в мордовии за 3 года выросла почти на треть', 0.20759109365555029), ('биткоин растет после падения, торгуясь у отметки в $7,5 тысячи', 0.22013362790365265), ('недельная ставка roisfix в четверг выросла до 10,08%', 0.23256556838366937), ('тамбовские сельхозкооперативы получили 73,5 млн рублей господдержки', 0.23336235587499266), ('на минобрнауки в 2018 году выделят более 400 миллиардов рублей', 0.2488325072511719), ('минэкономразвития: сумма соглашений на экспо в харбине превысит $1 миллиард', 0.25142158856605867), ('в 35 регионах россии превышены недельные эпидпороги по гриппу и орви', 0.25160665022631523), ('минфин прогнозирует рост заработной платы россиян', 0.25353194376588539), ('мурманская область ежегодно выделяет 1,6 млрд рублей на соцподдержку семей', 0.25369348667321279),

In [23]:
for clstr in range(NUM_CLUSTERS):
    cluster_to_topics(clstr)

[('россия', 21), ('рубль', 18), ('миллиард', 15), ('миллион', 9), ('2017', 8)]
[('задержать', 26), ('мужчина', 15), ('арестовать', 9), ('подозревать', 9), ('полиция', 9)]
[('теракт', 17), ('взрыв', 14), ('петербург', 12), ('метро', 11), ('сми', 9)]
[('сша', 29), ('заявить', 25), ('россия', 25), ('трамп', 22), ('путин', 15)]
[('выборы', 22), ('президент', 21), ('выбор', 18), ('франция', 15), ('макрон', 12)]
[('путин', 24), ('рассказать', 12), ('2017', 6), ('встреча', 6), ('нефть', 6)]
[('корея', 14), ('россия', 13), ('кндр', 12), ('южный', 12), ('путин', 11)]
[('сми', 11), ('сообщить', 11), ('сша', 9), ('боевик', 8), ('кндр', 8)]
[('трамп', 21), ('назвать', 11), ('россия', 10), ('путин', 10), ('встреча', 9)]
[('москва', 20), ('мчс', 10), ('самолёт', 10), ('рейс', 7), ('рассказать', 6)]
[('суд', 34), ('дело', 23), ('экс', 15), ('арест', 9), ('приговор', 8)]
[('ребёнок', 15), ('москва', 13), ('полицейский', 10), ('мужчина', 9), ('область', 8)]
[('российский', 17), ('россия', 12), ('сша', 

In [25]:
#25 кластеров, 25 повторений
for clstr in range(NUM_CLUSTERS):
    n = round(len(get_titles_by_cluster(clstr))/3)
    print(nClosestToCentroid(clstr, n))
    print('####\n')

[('неэффективный бизнес "отомрет" из-за низкой инфляции, считает орешкин', 0.17428300822859355), ('минфин прогнозирует рост заработной платы россиян', 0.18655931737935494), ('мешает геополитика: стоит ли ждать повышения рейтинга россии от s&p', 0.18784630017984294), ('"опора россии" ожидает дальнейшего снижения ключевой ставки цб', 0.20038959709415105), ('мэа повысило прогноз-2017 по добыче нефти в россии', 0.21175550376992769), ('российской экономике предрекли рост на уровне развитых стран', 0.21552508887740807), ('сбербанк не ожидает отрицательной доходности по вкладам, заявил греф', 0.2209779033863386), ('топилин назвал потери внебюджетных соцфондов при снижении страховых взносов', 0.2217510137916755), ('набиуллина не увидела минусов в текущей низкой инфляции в россии', 0.22343163915477271), ('цены на бензин выросли до рекордного уровня', 0.22844264778479961), ("глава минфина объяснил решение moody's улучшить прогноз по рейтингу россии", 0.23000451126580401), ('в минэкономразвития р

In [26]:
for clstr in range(NUM_CLUSTERS):
    cluster_to_topics(clstr)

[('россия', 13), ('рассказать', 12), ('считать', 11), ('рост', 10), ('нефть', 10)]
[('рубль', 20), ('миллиард', 14), ('миллион', 11), ('доллар', 7), ('евро', 7)]
[('задержать', 18), ('полиция', 15), ('москва', 10), ('полицейский', 9), ('подозревать', 8)]
[('колония', 6), ('тюрьма', 5), ('чиновник', 4), ('получить', 4), ('рассказать', 4)]
[('сша', 22), ('заявить', 20), ('россия', 19), ('считать', 16), ('путин', 15)]
[('путин', 24), ('встреча', 20), ('мид', 14), ('сша', 11), ('встретиться', 10)]
[('президент', 26), ('выборы', 23), ('выбор', 15), ('франция', 13), ('глава', 11)]
[('сми', 26), ('трамп', 9), ('британский', 9), ('сообщить', 8), ('американский', 6)]
[('россия', 7), ('пройти', 7), ('эксперт', 5), ('оценить', 5), ('фестиваль', 5)]
[('москва', 10), ('произойти', 9), ('землетрясение', 9), ('неделя', 8), ('магнитуда', 8)]
[('москва', 26), ('метро', 8), ('самолёт', 7), ('закрыть', 7), ('эвакуировать', 6)]
[('ребёнок', 18), ('мужчина', 15), ('полицейский', 11), ('задержать', 8), ('во

In [28]:
#35 кластеров, 25 повторений
for clstr in range(NUM_CLUSTERS):
    n = round(len(get_titles_by_cluster(clstr))/3)
    print(nClosestToCentroid(clstr, n))
    print('####\n')

[('у неработающей москвички украли спорткар за 12 миллионов рублей', 0.163090262430581), ('доход дмитрия пескова составил 12,8 млн рублей, татьяны навки - 120,8 млн', 0.17250480711446958), ('тамбовские сельхозкооперативы получили 73,5 млн рублей господдержки', 0.19335078423233532), ('курс евро к доллару обновил максимум с ноября 2016 года', 0.20815876990449556), ('в ставрополе полицейского оштрафовали на 11 млн рублей за взятку', 0.22132191542649948), ('напавшие на машину чоп в оренбурге похитили более шести миллионов рублей', 0.23337996670003691), ('мурманская область ежегодно выделяет 1,6 млрд рублей на соцподдержку семей', 0.23643738583014162), ('роскосмос отсудил у центра хруничева больше трех миллиардов рублей', 0.24708769035880096), ('арабская компания вложила в российские проекты полтора миллиарда долларов', 0.24859654584511048), ('украина рискует потерять три миллиарда долларов из-за бездействия чиновников', 0.25336159840358441), ('ивановская область получит более 10 млн рублей

In [29]:
for clstr in range(NUM_CLUSTERS):
    cluster_to_topics(clstr)

[('рубль', 19), ('миллиард', 12), ('миллион', 8), ('евро', 8), ('доллар', 7)]
[('терроризм', 7), ('поддержка', 7), ('борьба', 6), ('иго', 5), ('террорист', 5)]
[('россия', 17), ('нефть', 11), ('рост', 10), ('ожидать', 7), ('рынок', 6)]
[('назвать', 16), ('каталония', 8), ('действие', 4), ('сми', 3), ('эксперт', 3)]
[('военный', 7), ('турция', 6), ('саудовский', 5), ('аравия', 5), ('задержать', 5)]
[('сми', 5), ('рассказать', 5), ('задержать', 4), ('сын', 4), ('обвинить', 3)]
[('москва', 14), ('метро', 9), ('полиция', 8), ('петербург', 7), ('минирование', 7)]
[('выборы', 22), ('президент', 19), ('выбор', 14), ('франция', 11), ('макрон', 10)]
[('глава', 18), ('пост', 10), ('экс', 10), ('министр', 10), ('президент', 8)]
[('взрыв', 10), ('теракт', 9), ('пострадавший', 7), ('выразить', 5), ('соболезнование', 5)]
[('путин', 25), ('встреча', 18), ('россия', 13), ('мид', 11), ('встретиться', 11)]
[('задержать', 11), ('теракт', 9), ('дело', 8), ('убийство', 8), ('подозревать', 6)]
[('сша', 15),

In [5]:
#25 кластеров, 30 повторений
for clstr in range(NUM_CLUSTERS):
    n = round(len(get_titles_by_cluster(clstr))/3)
    print(nClosestToCentroid(clstr, n))
    print('####\n')

[('турция сообщила о нейтрализации почти 1000 боевиков за операцию в африне', 0.17154522959628615), ('в афганистане погибли восемь полицейских при нападении на кпп, сообщили сми', 0.21316962718608889), ('боевики иг* в мосуле потеряли централизованное руководство, заявили в ираке', 0.23743982733237234), ('армия ирака вступила на территорию старого города в мосуле', 0.23866319243799328), ('иракская армия начала операцию против боевиков иг* на западе страны', 0.24762595010735167), ('в сша скончалась обладательница нескольких "эмми" мэри тайлер мур', 0.26978909851440347), ('армия ирака освободила район мосула, где находится генконсульство турции', 0.27607053896112432), ('власти мьянмы предупредили об ответственности за поддержку боевиков', 0.28269242024233299), ('ливийская армия освободила от террористов юг страны и район триполи', 0.28277908129420704), ('иранские военные ликвидировали нескольких боевиков иг* на западе страны', 0.2859620625973931), ('воевавший в сирии боевик из дагестана э

In [6]:
for clstr in range(NUM_CLUSTERS):
    cluster_to_topics(clstr)

[('сообщить', 17), ('сми', 16), ('ирак', 10), ('турция', 8), ('боевик', 8)]
[('задержать', 20), ('теракт', 16), ('убийство', 11), ('полиция', 10), ('подозревать', 9)]
[('рубль', 22), ('миллиард', 18), ('5', 13), ('миллион', 10), ('доллар', 9)]
[('путин', 28), ('встреча', 17), ('глава', 15), ('президент', 14), ('встретиться', 13)]
[('нефть', 13), ('россия', 11), ('рост', 10), ('рассказать', 10), ('уровень', 8)]
[('выборы', 23), ('президент', 18), ('выбор', 15), ('франция', 11), ('макрон', 10)]
[('сша', 27), ('заявить', 26), ('россия', 23), ('путин', 16), ('рассказать', 11)]
[('москва', 18), ('взрыв', 11), ('метро', 11), ('полиция', 11), ('сообщить', 7)]
[('россиянин', 17), ('считать', 12), ('россия', 11), ('показать', 9), ('эксперт', 8)]
[('ребёнок', 19), ('мужчина', 12), ('полицейский', 10), ('водитель', 9), ('москва', 8)]
[('суд', 23), ('дело', 15), ('решение', 7), ('приговор', 6), ('арест', 6)]
[('экс', 20), ('глава', 20), ('министр', 9), ('суд', 8), ('дело', 7)]
[('область', 22), ('

In [8]:
#35 кластеров, 30 повторений
for clstr in range(NUM_CLUSTERS):
    n = round(len(get_titles_by_cluster(clstr))/3)
    print(nClosestToCentroid(clstr, n))
    print('####\n')

[('мэа повысило прогноз-2017 по добыче нефти в россии', 0.17539732269494024), ('недельная ставка roisfix в четверг выросла до 10,08%', 0.21044693093348765), ('цены на бензин выросли до рекордного уровня', 0.2135429281913499), ('набиуллина не увидела минусов в текущей низкой инфляции в россии', 0.21824571940956339), ('цены на нефть незначительно снизились после резкого роста', 0.22784902470079516), ('"победа" стала мировым лидером по темпам роста выручки среди лоукостеров', 0.23373268420004811), ('минэнерго в ближайшее время ожидает восстановления цен на нефть', 0.234335579845251), ('волков: урожайность зерновых в мордовии за 3 года выросла почти на треть', 0.23546724049247048), ('в глубокий минус. как схлопывается биткоин-пузырь', 0.23615770036936623), ('москва поднялась на 15 позиций в рейтинге самых дорогих городов мира', 0.23936330823115004), ('в минэкономразвития рассказали об ожидаемой инфляции в россии в сентябре', 0.25393009521425292), ('товарооборот россии и италии вырос на 28%

In [9]:
for clstr in range(NUM_CLUSTERS):
    cluster_to_topics(clstr)

[('россия', 9), ('рост', 8), ('нефть', 8), ('2017', 7), ('вырасти', 7)]
[('рубль', 22), ('миллиард', 15), ('миллион', 10), ('доллар', 6), ('1', 6)]
[('задержать', 17), ('теракт', 13), ('подозревать', 7), ('полицейский', 7), ('полиция', 7)]
[('назвать', 9), ('слово', 5), ('рассказать', 5), ('заявить', 5), ('путин', 4)]
[('рассказать', 5), ('россиянин', 5), ('российский', 5), ('россия', 5), ('трамп', 4)]
[('россия', 18), ('мид', 13), ('ес', 13), ('сша', 13), ('заявить', 12)]
[('рассказать', 9), ('роснефть', 5), ('система', 5), ('рынок', 5), ('нефть', 5)]
[('москва', 17), ('метро', 8), ('станция', 7), ('петербург', 6), ('звонок', 6)]
[('мужчина', 15), ('полицейский', 10), ('задержать', 9), ('ребёнок', 7), ('водитель', 5)]
[('боевик', 9), ('турция', 7), ('иго', 7), ('ирак', 7), ('заявить', 6)]
[('суд', 26), ('дело', 12), ('приговор', 7), ('решение', 6), ('арест', 5)]
[('министр', 13), ('отставка', 11), ('глава', 9), ('президент', 8), ('пост', 8)]
[('сша', 17), ('встреча', 13), ('мид', 13),

In [15]:
#35 кластеров, 20 повторений
for clstr in range(NUM_CLUSTERS):
    n = round(len(get_titles_by_cluster(clstr))/3)
    print(str(clstr)+'\t', nClosestToCentroid(clstr, n))
    print('####\n')

0	 [('в каталонии задержали мужчину, ранившего полицейских', 0.14698683472591556), ('в москве арестовали мужчину, обвиняемого в изнасиловании девочки в подъезде', 0.16682536250271818), ('в нижнем новгороде мужчину подозревают в убийства соседа, нагрубившего его бабушке', 0.17747970462095719), ('в петербурге задержали подростка, стрелявшего из пневматики по трамваю', 0.17981296384948198), ('одного из задержанных в ходе расследования теракта в манчестере освободили', 0.19579815396133216), ('задержанный по подозрению в подготовке теракта в вене признал связь с иг', 0.20175297506031076), ('в зеленограде мужчина устроил стрельбу в кафе', 0.20240975055330346), ('в петербурге задержали организаторов тренингов из-за самоубийства участницы', 0.20613291640880627), ('в мексике задержали предполагаемого убийцу модели-иностранки', 0.21098254750655565), ('в чечне торговцев оружием подозревают в связях с террористами', 0.21178258065621003), ('в балашихе арестовали мужчину, агитировавшего в соцсети за

In [17]:
for clstr in range(NUM_CLUSTERS):
    print(str(clstr))
    cluster_to_topics(clstr)

0	
[('задержать', 17), ('мужчина', 10), ('подозревать', 9), ('теракт', 9), ('арестовать', 7)]
1	
[('рубль', 21), ('миллиард', 15), ('миллион', 11), ('5', 6), ('доллар', 6)]
2	
[('россия', 12), ('нефть', 9), ('рынок', 7), ('считать', 7), ('рост', 6)]
3	
[('москва', 7), ('полиция', 7), ('глава', 6), ('задержать', 5), ('обвинить', 4)]
4	
[('россия', 5), ('победитель', 3), ('конкурс', 3), ('путин', 3), ('ленин', 3)]
5	
[('проверять', 6), ('минирование', 6), ('сообщение', 6), ('эвакуировать', 5), ('звонок', 5)]
6	
[('выборы', 20), ('президент', 14), ('выбор', 11), ('франция', 9), ('макрон', 8)]
7	
[('боевик', 9), ('сми', 7), ('ирак', 7), ('заявить', 6), ('иго', 6)]
8	
[('рассказать', 19), ('заявить', 10), ('путин', 9), ('изменение', 5), ('работа', 5)]
9	
[('путин', 16), ('встреча', 11), ('переговоры', 7), ('оон', 6), ('лавров', 6)]
10	
[('борьба', 11), ('обсудить', 11), ('россия', 9), ('турция', 7), ('сотрудничество', 7)]
11	
[('москва', 9), ('неделя', 7), ('ливень', 5), ('объявить', 5), ('

In [43]:
indxs = [6,7,12,16,20,21,24,30,33,34]
all_polemics = []
for indx in indxs:
    arr = []
    n = len(get_titles_by_cluster(indx))
    head_dists = nClosestToCentroid(indx, n)
    for head in head_dists:
        arr.append(head[0])
    all_polemics.append((indx, arr))

In [49]:
a=0
for topic in all_polemics:
    print(topic[0],len(topic[1]))

6 60
7 52
12 59
16 59
20 79
21 107
24 44
30 104
33 86
34 67


In [46]:
with open('polemic_headlines_by_themes.txt', 'w', encoding='utf-8') as f:
    for topic in all_polemics:
        f.write(str(topic[0])+'\n')
        for headline in topic[1]:
            f.write(headline+'\n')
        f.write('\n')

In [52]:
with open('polemic_headlines_by_themes.txt', 'r', encoding='utf-8') as f1:
    topics = f1.read().split('\n\n')

In [53]:
len(topics)

10

In [55]:
for topic in topics:
    topic = topic.split('\n')
    print(topic[0], len(topic[1:]))

6 18
7 28
12 13
16 7
20 32
21 20
24 8
30 68
33 17
34 17


In [57]:
with open('all_heads_links_nodups.txt', 'r', encoding='utf-8') as f1:
    just_all = f1.read().split('\n')

JSONDecodeError: Expecting value: line 1 column 2 (char 1)